# **Photogallery and Photographer's event**

## **Introduction**

A company based in Toronto, Canada is organising an event, for photographers and their captured moments from all over the world. The organisers are willing to make a good impression, memoriable stay of guest at hotel, conference or discussion hall, 
local landscape to visit, stores for shopping, restaurants and cafes nearby. So the organiser's purpose is to make a list of all possible landscapes in Toronto, including nearest restaurants, cafes, and shopping malls for every guest.

## **Data Description**

The data used in this project is provided by Foursquare location data. Data is grouped by landscape area, and each entry in the dataframe included information on area, restaurants, cafes, and stores within the vicinity of area.

## **Table of contents**

#### 01- Import Libraries
#### 02- Define Foursquare Credentials
#### 03- Define the city and get its latitude & longitude
#### 04- Search for Hotels & clean dataframe
#### 05- Search for Parks & clean dataframe
#### 06- Search for Restaurants & clean dataframe
#### 07- Search for Cafeteria & clean dataframe 
#### 08- Search for Shopping Stores & clean dataframe
#### 09- Generate map to visualize hotels, shopping stores and Cafeteria and how they cluster together
#### 10- Generate map to visualize Park, Restaurant and Cafeteria and how they cluster together

## **Import Libraries**

In [3]:
import requests # to handle requests
import pandas as pd # for data analsysis
import numpy as np # to handle data in a vectorized manner

!pip install geopy 
from geopy.geocoders import Nominatim # module to convert an address into latitude and longitude values

# libraries for displaying images
from IPython.display import Image 
from IPython.core.display import HTML 
    
#tranforming json file into a pandas dataframe library
from pandas.io.json import json_normalize

import folium # plotting library

## **Define Foursquare Credentials**

In [4]:
ClIENT_ID = 'TGM5K1WCBLTUMURGV1KRVXHNJERYE5WKFOZFW3ET2MP5OL1S' # your Foursquare ID
ClIENT_SECRET = 'A1Y33Z44WSTFEL4XASW20OMOWGIHQY1NO0QXXCTWUXP4ANPV' # your Foursquare Secret
VERSION = '20200104'
LIMIT =30
print('Your credentails:')
print('Foursquare_ID: ' + ClIENT_ID)
print('Foursquare_Secret:' + ClIENT_SECRET)

Your credentails:
Foursquare_ID: TGM5K1WCBLTUMURGV1KRVXHNJERYE5WKFOZFW3ET2MP5OL1S
Foursquare_Secret:A1Y33Z44WSTFEL4XASW20OMOWGIHQY1NO0QXXCTWUXP4ANPV


## **Define the city and get its latitude & longitude** 

In [6]:
# define the city and get its latitude & longitude 
city = 'Toronto'
geolocator = Nominatim(user_agent="foursquare_agent")
location = geolocator.geocode(city)
latitude = location.latitude
longitude = location.longitude
print(latitude, longitude)

43.653963 -79.387207


## **Search for Hotels**

In [7]:
# search for hotels
search_query = 'Hotel'
radius = 500

# Define the corresponding URL
url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'\
.format(ClIENT_ID, ClIENT_SECRET, latitude, longitude, VERSION, search_query, radius, LIMIT)
url

'https://api.foursquare.com/v2/venues/search?client_id=TGM5K1WCBLTUMURGV1KRVXHNJERYE5WKFOZFW3ET2MP5OL1S&client_secret=A1Y33Z44WSTFEL4XASW20OMOWGIHQY1NO0QXXCTWUXP4ANPV&ll=43.653963,-79.387207&v=20200104&query=Hotel&radius=500&limit=30'

In [0]:
# Send the GET Request and examine the results
results = requests.get(url).json()
#results

In [9]:
# assign relevant part of JSON to venues
venues = results['response']['venues']

# tranform venues into a dataframe
dataframe = json_normalize(venues)
dataframe.head()

,id,name,categories,referralId,hasPerk,location.address,location.crossStreet,location.lat,location.lng,location.labeledLatLngs,location.distance,location.postalCode,location.cc,location.city,location.state,location.country,location.formattedAddress,venuePage.id,location.neighborhood
0,4ab2d511f964a5209b6c20e3,Sheraton Centre Toronto Hotel,"[{'id': '4bf58dd8d48988d1fa931735', 'name': 'H...",v-1578117948,False,123 Queen Street West,at York St.,43.651144,-79.384329,"[{'label': 'display', 'lat': 43.6511441, 'lng'...",390,M5H 2M9,CA,Toronto,ON,Canada,"[123 Queen Street West (at York St.), Toronto ...",NaN,NaN
1,4b68aed1f964a520de862be3,The Rex Hotel Jazz & Blues Bar,"[{'id': '4bf58dd8d48988d1e7931735', 'name': 'J...",v-1578117948,False,194 Queen St W,Queen & St. Patrick,43.650505,-79.388577,"[{'label': 'display', 'lat': 43.65050475544005...",400,M5V 1Z1,CA,Toronto,ON,Canada,"[194 Queen St W (Queen & St. Patrick), Toronto...",62225795,NaN
2,52ce14b0498e50457ce11780,DoubleTree by Hilton Hotel Toronto Downtown,"[{'id': '4bf58dd8d48988d1fa931735', 'name': 'H...",v-1578117948,False,108 Chestnut Street,Dundas St W,43.654608,-79.385942,"[{'label': 'display', 'lat': 43.6546083, 'lng'...",124,M5G 1R3,CA,Toronto,ON,Canada,"[108 Chestnut Street (Dundas St W), Toronto ON...",NaN,NaN
3,4f343a31e4b0230a3b337a90,VFM Test Hotel,"[{'id': '4bf58dd8d48988d1fa931735', 'name': 'H...",v-1578117948,False,123 Test Drive,at somewhere St,43.658434,-79.387894,"[{'label': 'display', 'lat': 43.658434, 'lng':...",500,M2M 2M2,CA,Toronto,ON,Canada,"[123 Test Drive (at somewhere St), Toronto ON ...",NaN,NaN
4,5545d07e498e2facac03f666,Sheraton Centre Toronto Hotel - Grand Ballroom,"[{'id': '56aa371be4b08b9a8d5734cf', 'name': 'B...",v-1578117948,False,123 Queen Street West,at Bay Street,43.651200,-79.384520,"[{'label': 'display', 'lat': 43.65119976208405...",376,NaN,CA,Toronto,ON,Canada,"[123 Queen Street West (at Bay Street), Toront...",NaN,Financial District


## **Clean Hotel Dataframe**

In [10]:
# keep only columns that include venue name, and anything that is associated with location
clean_columns = ['name', 'categories'] + [col for col in dataframe.columns if col.startswith('location.')]+ ['id']
clean_dataframe = dataframe.loc[:,clean_columns]

# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

# filter the category for each row
clean_dataframe['categories'] = clean_dataframe.apply(get_category_type, axis=1)

# clean column names by keeping only last term
clean_dataframe.columns = [column.split('.')[-1] for column in clean_dataframe.columns]

clean_dataframe.head()

,name,categories,address,crossStreet,lat,lng,labeledLatLngs,distance,postalCode,cc,city,state,country,formattedAddress,neighborhood,id
0,Sheraton Centre Toronto Hotel,Hotel,123 Queen Street West,at York St.,43.651144,-79.384329,"[{'label': 'display', 'lat': 43.6511441, 'lng'...",390,M5H 2M9,CA,Toronto,ON,Canada,"[123 Queen Street West (at York St.), Toronto ...",NaN,4ab2d511f964a5209b6c20e3
1,The Rex Hotel Jazz & Blues Bar,Jazz Club,194 Queen St W,Queen & St. Patrick,43.650505,-79.388577,"[{'label': 'display', 'lat': 43.65050475544005...",400,M5V 1Z1,CA,Toronto,ON,Canada,"[194 Queen St W (Queen & St. Patrick), Toronto...",NaN,4b68aed1f964a520de862be3
2,DoubleTree by Hilton Hotel Toronto Downtown,Hotel,108 Chestnut Street,Dundas St W,43.654608,-79.385942,"[{'label': 'display', 'lat': 43.6546083, 'lng'...",124,M5G 1R3,CA,Toronto,ON,Canada,"[108 Chestnut Street (Dundas St W), Toronto ON...",NaN,52ce14b0498e50457ce11780
3,VFM Test Hotel,Hotel,123 Test Drive,at somewhere St,43.658434,-79.387894,"[{'label': 'display', 'lat': 43.658434, 'lng':...",500,M2M 2M2,CA,Toronto,ON,Canada,"[123 Test Drive (at somewhere St), Toronto ON ...",NaN,4f343a31e4b0230a3b337a90
4,Sheraton Centre Toronto Hotel - Grand Ballroom,Ballroom,123 Queen Street West,at Bay Street,43.651200,-79.384520,"[{'label': 'display', 'lat': 43.65119976208405...",376,NaN,CA,Toronto,ON,Canada,"[123 Queen Street West (at Bay Street), Toront...",Financial District,5545d07e498e2facac03f666


In [11]:
# delete unnecessary columns
clean_dataframe2= clean_dataframe.drop(['cc', 'city', 'country', 'crossStreet', 'distance', 'formattedAddress',\
                                        'labeledLatLngs','neighborhood', 'id'], axis=1)
clean_dataframe2

,name,categories,address,lat,lng,postalCode,state
0,Sheraton Centre Toronto Hotel,Hotel,123 Queen Street West,43.651144,-79.384329,M5H 2M9,ON
1,The Rex Hotel Jazz & Blues Bar,Jazz Club,194 Queen St W,43.650505,-79.388577,M5V 1Z1,ON
2,DoubleTree by Hilton Hotel Toronto Downtown,Hotel,108 Chestnut Street,43.654608,-79.385942,M5G 1R3,ON
3,VFM Test Hotel,Hotel,123 Test Drive,43.658434,-79.387894,M2M 2M2,ON
4,Sheraton Centre Toronto Hotel - Grand Ballroom,Ballroom,123 Queen Street West,43.651200,-79.384520,NaN,ON
5,Stathcona Hotel,Hotel,NaN,43.654947,-79.386359,NaN,ON
6,Sheraton Centre Hotel Club Lounge,Lounge,NaN,43.651063,-79.384527,M5H 2M9,ON
7,650 Hotel,Hotel,650 Bay Street,43.657046,-79.384411,NaN,ON
8,Be SixFifty Hotel,Hotel,650 Bay Street,43.657120,-79.384560,NaN,ON
9,Shangri-La Toronto,Hotel,188 University Ave.,43.649129,-79.386557,M5H 0A3,ON


In [12]:
# delete rows with none values
clean_dataframe3 = clean_dataframe2.dropna(axis=0, how='any', thresh=None, subset=None, inplace=False)
clean_dataframe3

,name,categories,address,lat,lng,postalCode,state
0,Sheraton Centre Toronto Hotel,Hotel,123 Queen Street West,43.651144,-79.384329,M5H 2M9,ON
1,The Rex Hotel Jazz & Blues Bar,Jazz Club,194 Queen St W,43.650505,-79.388577,M5V 1Z1,ON
2,DoubleTree by Hilton Hotel Toronto Downtown,Hotel,108 Chestnut Street,43.654608,-79.385942,M5G 1R3,ON
3,VFM Test Hotel,Hotel,123 Test Drive,43.658434,-79.387894,M2M 2M2,ON
9,Shangri-La Toronto,Hotel,188 University Ave.,43.649129,-79.386557,M5H 0A3,ON
10,Hilton Toronto,Hotel,145 Richmond St W,43.649946,-79.385479,M5H 2L2,ON
11,89 Chestnut Residence,College Residence Hall,89 Chestnut St.,43.654160,-79.385291,M5G 1R1,ON
12,Marriott Downtown at CF Toronto Eaton Centre,Hotel,525 Bay Street,43.654728,-79.382422,M5G 2L2,ON
13,Lobby Lounge at the Shangri-La Toronto,Lounge,188 University Ave.,43.649155,-79.386546,M5H 0A3,ON
15,Fitness Centre,Gym,Sheraton Centre,43.650985,-79.384002,M5H 2M9,ON


In [13]:
# delete rows which its category is not Hotel or Event Space
array= ['Hotel', 'Event Space']
hotel_dataframe= clean_dataframe3.loc[clean_dataframe3['categories'].isin(array)]
hotel_dataframe

,name,categories,address,lat,lng,postalCode,state
0,Sheraton Centre Toronto Hotel,Hotel,123 Queen Street West,43.651144,-79.384329,M5H 2M9,ON
2,DoubleTree by Hilton Hotel Toronto Downtown,Hotel,108 Chestnut Street,43.654608,-79.385942,M5G 1R3,ON
3,VFM Test Hotel,Hotel,123 Test Drive,43.658434,-79.387894,M2M 2M2,ON
9,Shangri-La Toronto,Hotel,188 University Ave.,43.649129,-79.386557,M5H 0A3,ON
10,Hilton Toronto,Hotel,145 Richmond St W,43.649946,-79.385479,M5H 2L2,ON
12,Marriott Downtown at CF Toronto Eaton Centre,Hotel,525 Bay Street,43.654728,-79.382422,M5G 2L2,ON
17,Grand Ballroom,Event Space,123 Queen St. W,43.651217,-79.383771,M5H 2M9,ON


In [14]:
# delete rows which has duplicate hotel's name
df_hotels = hotel_dataframe.drop_duplicates(subset='name', keep="first")
df_hotels

,name,categories,address,lat,lng,postalCode,state
0,Sheraton Centre Toronto Hotel,Hotel,123 Queen Street West,43.651144,-79.384329,M5H 2M9,ON
2,DoubleTree by Hilton Hotel Toronto Downtown,Hotel,108 Chestnut Street,43.654608,-79.385942,M5G 1R3,ON
3,VFM Test Hotel,Hotel,123 Test Drive,43.658434,-79.387894,M2M 2M2,ON
9,Shangri-La Toronto,Hotel,188 University Ave.,43.649129,-79.386557,M5H 0A3,ON
10,Hilton Toronto,Hotel,145 Richmond St W,43.649946,-79.385479,M5H 2L2,ON
12,Marriott Downtown at CF Toronto Eaton Centre,Hotel,525 Bay Street,43.654728,-79.382422,M5G 2L2,ON
17,Grand Ballroom,Event Space,123 Queen St. W,43.651217,-79.383771,M5H 2M9,ON


In [15]:
# choose the hotel which has the same postalCode with the event space
df_hotel = df_hotels[df_hotels.postalCode == 'M5H 2M9']
df_hotel

,name,categories,address,lat,lng,postalCode,state
0,Sheraton Centre Toronto Hotel,Hotel,123 Queen Street West,43.651144,-79.384329,M5H 2M9,ON
17,Grand Ballroom,Event Space,123 Queen St. W,43.651217,-79.383771,M5H 2M9,ON


## **Search for Parks**

In [16]:
# search for Parks
search_query = 'Park'
radius = 10000

# Define the corresponding URL
url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'\
.format(ClIENT_ID, ClIENT_SECRET, latitude, longitude, VERSION, search_query, radius, LIMIT)
url

'https://api.foursquare.com/v2/venues/search?client_id=TGM5K1WCBLTUMURGV1KRVXHNJERYE5WKFOZFW3ET2MP5OL1S&client_secret=A1Y33Z44WSTFEL4XASW20OMOWGIHQY1NO0QXXCTWUXP4ANPV&ll=43.653963,-79.387207&v=20200104&query=Park&radius=10000&limit=30'

In [0]:
# Send the GET Request and examine the results
presults = requests.get(url).json()
#presults

In [18]:
# assign relevant part of JSON to venues
venues = presults['response']['venues']

# tranform venues into a dataframe
park_dataframe = json_normalize(venues)
park_dataframe.head()

,id,name,categories,referralId,hasPerk,location.address,location.crossStreet,location.lat,location.lng,location.labeledLatLngs,location.distance,location.postalCode,location.cc,location.city,location.state,location.country,location.formattedAddress,venuePage.id
0,4b170e47f964a5208cc123e3,Queen's Park Subway Station,"[{'id': '4bf58dd8d48988d1fd931735', 'name': 'M...",v-1578117992,False,691 University Ave,at College St,43.660006,-79.390272,"[{'label': 'display', 'lat': 43.66000635700919...",716,M5G 2P1,CA,Toronto,ON,Canada,"[691 University Ave (at College St), Toronto O...",NaN
1,4b54deadf964a520a3d027e3,Grange Park,"[{'id': '4bf58dd8d48988d163941735', 'name': 'P...",v-1578117992,False,NaN,NaN,43.652488,-79.392053,"[{'label': 'display', 'lat': 43.65248755299856...",423,NaN,CA,Toronto,ON,Canada,"[btwn College, University, Queen & Spadina, To...",NaN
2,4d3a1d23325ff04db2dd1845,Residences at College Park North,"[{'id': '4d954b06a243a5684965b473', 'name': 'R...",v-1578117992,False,763 Bay St.,btwn. College St. and Gerrard St.,43.659822,-79.385159,"[{'label': 'display', 'lat': 43.65982157793473...",672,M5G 2R3,CA,Toronto,ON,Canada,[763 Bay St. (btwn. College St. and Gerrard St...,NaN
3,4bb79860cf2fc9b616779e02,Berczy Park,"[{'id': '4bf58dd8d48988d163941735', 'name': 'P...",v-1578117992,False,35 Wellington St. East,NaN,43.648048,-79.375172,"[{'label': 'display', 'lat': 43.64804835388851...",1171,NaN,CA,Toronto,ON,Canada,"[35 Wellington St. East, Toronto ON, Canada]",NaN
4,4ad80fcef964a520a61021e3,One Park Lane,"[{'id': '4d954b06a243a5684965b473', 'name': 'R...",v-1578117992,False,280 Simcoe St,Dundas Street West,43.655442,-79.390056,"[{'label': 'display', 'lat': 43.65544241747622...",282,M5T 2Y5,CA,Toronto,ON,Canada,"[280 Simcoe St (Dundas Street West), Toronto O...",NaN


## **Clean Park Dataframe**

In [19]:
# keep only columns that include venue name, and anything that is associated with location
park_clean_columns = ['name', 'categories'] + [col for col in park_dataframe.columns if col.startswith('location.')]+ ['id']
clean_park_dataframe = park_dataframe.loc[:,park_clean_columns]

# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list1 = row['categories']
    except:
        categories_list1 = row['venue.categories']
        
    if len(categories_list1) == 0:
        return None
    else:
        return categories_list1[0]['name']

# filter the category for each row
clean_park_dataframe['categories'] = clean_park_dataframe.apply(get_category_type, axis=1)

# clean column names by keeping only last term
clean_park_dataframe.columns = [column.split('.')[-1] for column in clean_park_dataframe.columns]

clean_park_dataframe.head()

,name,categories,address,crossStreet,lat,lng,labeledLatLngs,distance,postalCode,cc,city,state,country,formattedAddress,id
0,Queen's Park Subway Station,Metro Station,691 University Ave,at College St,43.660006,-79.390272,"[{'label': 'display', 'lat': 43.66000635700919...",716,M5G 2P1,CA,Toronto,ON,Canada,"[691 University Ave (at College St), Toronto O...",4b170e47f964a5208cc123e3
1,Grange Park,Park,NaN,NaN,43.652488,-79.392053,"[{'label': 'display', 'lat': 43.65248755299856...",423,NaN,CA,Toronto,ON,Canada,"[btwn College, University, Queen & Spadina, To...",4b54deadf964a520a3d027e3
2,Residences at College Park North,Residential Building (Apartment / Condo),763 Bay St.,btwn. College St. and Gerrard St.,43.659822,-79.385159,"[{'label': 'display', 'lat': 43.65982157793473...",672,M5G 2R3,CA,Toronto,ON,Canada,[763 Bay St. (btwn. College St. and Gerrard St...,4d3a1d23325ff04db2dd1845
3,Berczy Park,Park,35 Wellington St. East,NaN,43.648048,-79.375172,"[{'label': 'display', 'lat': 43.64804835388851...",1171,NaN,CA,Toronto,ON,Canada,"[35 Wellington St. East, Toronto ON, Canada]",4bb79860cf2fc9b616779e02
4,One Park Lane,Residential Building (Apartment / Condo),280 Simcoe St,Dundas Street West,43.655442,-79.390056,"[{'label': 'display', 'lat': 43.65544241747622...",282,M5T 2Y5,CA,Toronto,ON,Canada,"[280 Simcoe St (Dundas Street West), Toronto O...",4ad80fcef964a520a61021e3


In [20]:
# delete unnecessary columns
clean_park_dataframe2= clean_park_dataframe.drop(['cc', 'city', 'country', 'crossStreet', 'distance', 'formattedAddress',\
                                        'labeledLatLngs', 'id'], axis=1)
clean_park_dataframe2

,name,categories,address,lat,lng,postalCode,state
0,Queen's Park Subway Station,Metro Station,691 University Ave,43.660006,-79.390272,M5G 2P1,ON
1,Grange Park,Park,NaN,43.652488,-79.392053,NaN,ON
2,Residences at College Park North,Residential Building (Apartment / Condo),763 Bay St.,43.659822,-79.385159,M5G 2R3,ON
3,Berczy Park,Park,35 Wellington St. East,43.648048,-79.375172,NaN,ON
4,One Park Lane,Residential Building (Apartment / Condo),280 Simcoe St,43.655442,-79.390056,M5T 2Y5,ON
5,St. Andrews Playground / Dog Park,Park,450 Adelaide Street West,43.647388,-79.398752,NaN,ON
6,Queen's Park,Park,University Ave.,43.663946,-79.392180,M5R 2E8,ON
7,TTC Streetcar #403 - Victoria park,Light Rail Station,NaN,43.650735,-79.386921,NaN,ON
8,Crunch Fitness - College Park,Gym / Fitness Center,382 Yonge Street,43.659263,-79.382632,M5G 1S8,ON
9,Clarence Square Park,Park,Spadina Ave,43.644222,-79.394230,NaN,ON


In [21]:
# delete rows with none values
clean_park_dataframe3 = clean_park_dataframe2.dropna(axis=0, how='any', thresh=None, subset=None, inplace=False)
clean_park_dataframe3

,name,categories,address,lat,lng,postalCode,state
0,Queen's Park Subway Station,Metro Station,691 University Ave,43.660006,-79.390272,M5G 2P1,ON
2,Residences at College Park North,Residential Building (Apartment / Condo),763 Bay St.,43.659822,-79.385159,M5G 2R3,ON
4,One Park Lane,Residential Building (Apartment / Condo),280 Simcoe St,43.655442,-79.390056,M5T 2Y5,ON
6,Queen's Park,Park,University Ave.,43.663946,-79.392180,M5R 2E8,ON
8,Crunch Fitness - College Park,Gym / Fitness Center,382 Yonge Street,43.659263,-79.382632,M5G 1S8,ON
10,Physiomed College Park,Medical Center,382 Yonge St,43.659240,-79.382860,M5B 1S8,ON
11,Trinity Bellwoods Park,Park,1053 Dundas St. W.,43.647072,-79.413756,M5H 2N2,ON
14,Bellevue Square Park,Park,btwn Bellevue & Augusta Ave.,43.653610,-79.402199,M5T 2N4,ON
16,College Park,Shopping Mall,444 Yonge St,43.661237,-79.383603,M5B 2H4,ON
18,Canoe Landing Park,Park,50 Fort York Blvd,43.638762,-79.397067,M5V 3Z1,ON


In [22]:
# delete rows which its category is not Park
df_park = clean_park_dataframe3[clean_park_dataframe3.categories == 'Park']
df_park

,name,categories,address,lat,lng,postalCode,state
6,Queen's Park,Park,University Ave.,43.663946,-79.392180,M5R 2E8,ON
11,Trinity Bellwoods Park,Park,1053 Dundas St. W.,43.647072,-79.413756,M5H 2N2,ON
14,Bellevue Square Park,Park,btwn Bellevue & Augusta Ave.,43.653610,-79.402199,M5T 2N4,ON
18,Canoe Landing Park,Park,50 Fort York Blvd,43.638762,-79.397067,M5V 3Z1,ON
22,Riverdale Park East,Park,550 Broadview Ave,43.669951,-79.355493,M4K 2P1,ON
23,Barbara Hall Park,Park,519 Church St,43.666879,-79.381068,M4Y 2K9,ON


## **Search for Restaurants**

In [23]:
# search for Restaurants
search_query = 'Restaurant'
radius = 10000

# Define the corresponding URL
url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(ClIENT_ID, ClIENT_SECRET, latitude, longitude, VERSION, search_query, radius, LIMIT)
url

'https://api.foursquare.com/v2/venues/search?client_id=TGM5K1WCBLTUMURGV1KRVXHNJERYE5WKFOZFW3ET2MP5OL1S&client_secret=A1Y33Z44WSTFEL4XASW20OMOWGIHQY1NO0QXXCTWUXP4ANPV&ll=43.653963,-79.387207&v=20200104&query=Restaurant&radius=10000&limit=30'

In [0]:
# Send the GET Request and examine the results
Rresults = requests.get(url).json()
#Rresults

In [25]:
# assign relevant part of JSON to venues
venues = Rresults['response']['venues']

# tranform venues into a dataframe
Restaurant_dataframe = json_normalize(venues)
Restaurant_dataframe.head()

,id,name,categories,referralId,hasPerk,location.address,location.lat,location.lng,location.labeledLatLngs,location.distance,location.postalCode,location.cc,location.city,location.state,location.country,location.formattedAddress,location.crossStreet,location.neighborhood,venuePage.id
0,4ad4c05ff964a52048f720e3,Hemispheres Restaurant & Bistro,"[{'id': '4bf58dd8d48988d14e941735', 'name': 'A...",v-1578118031,False,110 Chestnut Street,43.654884,-79.385931,"[{'label': 'display', 'lat': 43.65488413420439...",145,M5G 1R3,CA,Toronto,ON,Canada,"[110 Chestnut Street, Toronto ON M5G 1R3, Canada]",NaN,NaN,NaN
1,4ad4c060f964a5207ff720e3,Rol San Restaurant 龍笙棧,"[{'id': '4bf58dd8d48988d1f5931735', 'name': 'D...",v-1578118031,False,323 Spadina Ave.,43.654318,-79.398650,"[{'label': 'display', 'lat': 43.65431754076345...",922,M5T 2E9,CA,Toronto,ON,Canada,"[323 Spadina Ave. (at D'Arcy St.), Toronto ON ...",at D'Arcy St.,Kensington Market,NaN
2,4ae29812f964a520288f21e3,Swatow Restaurant 汕頭小食家,"[{'id': '4bf58dd8d48988d145941735', 'name': 'C...",v-1578118031,False,309 Spadina Ave.,43.653866,-79.398334,"[{'label': 'display', 'lat': 43.65386562507761...",896,M5T 2E6,CA,Toronto,ON,Canada,[309 Spadina Ave. (btwn Dundas St. W & D'Arcy ...,btwn Dundas St. W & D'Arcy St.,NaN,NaN
3,4b074bb1f964a52077fb22e3,New Sky Restaurant 小沙田食家,"[{'id': '4bf58dd8d48988d145941735', 'name': 'C...",v-1578118031,False,353 Spadina Ave.,43.655337,-79.398897,"[{'label': 'display', 'lat': 43.65533674412141...",953,M5T 2G3,CA,Toronto,ON,Canada,"[353 Spadina Ave., Toronto ON M5T 2G3, Canada]",NaN,NaN,NaN
4,4b266f05f964a520657b24e3,Goldstone Noodle Restaurant 金石,"[{'id': '4bf58dd8d48988d1d1941735', 'name': 'N...",v-1578118031,False,266 Spadina Ave,43.652278,-79.398039,"[{'label': 'display', 'lat': 43.6522783893466,...",892,M5T 2E4,CA,Toronto,ON,Canada,"[266 Spadina Ave (at Willison Sq), Toronto ON ...",at Willison Sq,NaN,NaN


## **Clean Restaurant Dataframe**

In [26]:
# keep only columns that include venue name, and anything that is associated with location
Restaurant_clean_columns = ['name', 'categories'] + [col for col in Restaurant_dataframe.columns if col.startswith('location.')]+ ['id']
clean_Restaurant_dataframe = Restaurant_dataframe.loc[:,Restaurant_clean_columns]

# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list3 = row['categories']
    except:
        categories_list3 = row['venue.categories']
        
    if len(categories_list3) == 0:
        return None
    else:
        return categories_list3[0]['name']

# filter the category for each row
clean_Restaurant_dataframe['categories'] = clean_Restaurant_dataframe.apply(get_category_type, axis=1)

# clean column names by keeping only last term
clean_Restaurant_dataframe.columns = [column.split('.')[-1] for column in clean_Restaurant_dataframe.columns]

clean_Restaurant_dataframe.head()

,name,categories,address,lat,lng,labeledLatLngs,distance,postalCode,cc,city,state,country,formattedAddress,crossStreet,neighborhood,id
0,Hemispheres Restaurant & Bistro,American Restaurant,110 Chestnut Street,43.654884,-79.385931,"[{'label': 'display', 'lat': 43.65488413420439...",145,M5G 1R3,CA,Toronto,ON,Canada,"[110 Chestnut Street, Toronto ON M5G 1R3, Canada]",NaN,NaN,4ad4c05ff964a52048f720e3
1,Rol San Restaurant 龍笙棧,Dim Sum Restaurant,323 Spadina Ave.,43.654318,-79.398650,"[{'label': 'display', 'lat': 43.65431754076345...",922,M5T 2E9,CA,Toronto,ON,Canada,"[323 Spadina Ave. (at D'Arcy St.), Toronto ON ...",at D'Arcy St.,Kensington Market,4ad4c060f964a5207ff720e3
2,Swatow Restaurant 汕頭小食家,Chinese Restaurant,309 Spadina Ave.,43.653866,-79.398334,"[{'label': 'display', 'lat': 43.65386562507761...",896,M5T 2E6,CA,Toronto,ON,Canada,[309 Spadina Ave. (btwn Dundas St. W & D'Arcy ...,btwn Dundas St. W & D'Arcy St.,NaN,4ae29812f964a520288f21e3
3,New Sky Restaurant 小沙田食家,Chinese Restaurant,353 Spadina Ave.,43.655337,-79.398897,"[{'label': 'display', 'lat': 43.65533674412141...",953,M5T 2G3,CA,Toronto,ON,Canada,"[353 Spadina Ave., Toronto ON M5T 2G3, Canada]",NaN,NaN,4b074bb1f964a52077fb22e3
4,Goldstone Noodle Restaurant 金石,Noodle House,266 Spadina Ave,43.652278,-79.398039,"[{'label': 'display', 'lat': 43.6522783893466,...",892,M5T 2E4,CA,Toronto,ON,Canada,"[266 Spadina Ave (at Willison Sq), Toronto ON ...",at Willison Sq,NaN,4b266f05f964a520657b24e3


In [27]:
# delete unnecessary columns
clean_Restaurant_dataframe2= clean_Restaurant_dataframe.drop(['cc', 'city', 'country', 'crossStreet', 'distance', 'formattedAddress',\
                                        'labeledLatLngs', 'neighborhood', 'id'], axis=1)
clean_Restaurant_dataframe2

,name,categories,address,lat,lng,postalCode,state
0,Hemispheres Restaurant & Bistro,American Restaurant,110 Chestnut Street,43.654884,-79.385931,M5G 1R3,ON
1,Rol San Restaurant 龍笙棧,Dim Sum Restaurant,323 Spadina Ave.,43.654318,-79.398650,M5T 2E9,ON
2,Swatow Restaurant 汕頭小食家,Chinese Restaurant,309 Spadina Ave.,43.653866,-79.398334,M5T 2E6,ON
3,New Sky Restaurant 小沙田食家,Chinese Restaurant,353 Spadina Ave.,43.655337,-79.398897,M5T 2G3,ON
4,Goldstone Noodle Restaurant 金石,Noodle House,266 Spadina Ave,43.652278,-79.398039,M5T 2E4,ON
5,Aroma Fine Indian Restaurant,Indian Restaurant,287 King St. W,43.646463,-79.389644,M5V 1J5,ON
6,Victoria's Restaurant,Restaurant,37 King Street East,43.649298,-79.376431,M5C 1E9,ON
7,360 Restaurant,Wine Bar,301 Front St W,43.642537,-79.387042,M5V 2T6,ON
8,Azure Restaurant & Bar,Restaurant,225 Front St W,43.644749,-79.385113,M5V 2X3,ON
9,North-East Chinese Restaurant 華北美食,Chinese Restaurant,476 Dundas St.,43.653185,-79.396677,NaN,ON


In [28]:
# delete rows with none values
df_Restaurant = clean_Restaurant_dataframe2.dropna(axis=0, how='any', thresh=None, subset=None, inplace=False)
df_Restaurant

,name,categories,address,lat,lng,postalCode,state
0,Hemispheres Restaurant & Bistro,American Restaurant,110 Chestnut Street,43.654884,-79.385931,M5G 1R3,ON
1,Rol San Restaurant 龍笙棧,Dim Sum Restaurant,323 Spadina Ave.,43.654318,-79.398650,M5T 2E9,ON
2,Swatow Restaurant 汕頭小食家,Chinese Restaurant,309 Spadina Ave.,43.653866,-79.398334,M5T 2E6,ON
3,New Sky Restaurant 小沙田食家,Chinese Restaurant,353 Spadina Ave.,43.655337,-79.398897,M5T 2G3,ON
4,Goldstone Noodle Restaurant 金石,Noodle House,266 Spadina Ave,43.652278,-79.398039,M5T 2E4,ON
5,Aroma Fine Indian Restaurant,Indian Restaurant,287 King St. W,43.646463,-79.389644,M5V 1J5,ON
6,Victoria's Restaurant,Restaurant,37 King Street East,43.649298,-79.376431,M5C 1E9,ON
7,360 Restaurant,Wine Bar,301 Front St W,43.642537,-79.387042,M5V 2T6,ON
8,Azure Restaurant & Bar,Restaurant,225 Front St W,43.644749,-79.385113,M5V 2X3,ON
10,The Hot House Restaurant & Bar,American Restaurant,35 Church St,43.648824,-79.373702,M5E 1T3,ON


## **Search for Cafeteria**

In [29]:
# search for Cafeteria
search_query = 'Cafeteria'
radius = 10000

# Define the corresponding URL
url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(ClIENT_ID, ClIENT_SECRET, latitude, longitude, VERSION, search_query, radius, LIMIT)
url

'https://api.foursquare.com/v2/venues/search?client_id=TGM5K1WCBLTUMURGV1KRVXHNJERYE5WKFOZFW3ET2MP5OL1S&client_secret=A1Y33Z44WSTFEL4XASW20OMOWGIHQY1NO0QXXCTWUXP4ANPV&ll=43.653963,-79.387207&v=20200104&query=Cafeteria&radius=10000&limit=30'

In [0]:
# Send the GET Request and examine the results
cresults = requests.get(url).json()


In [32]:
# assign relevant part of JSON to venues
venues = cresults['response']['venues']

# tranform venues into a dataframe
Cafeteria_dataframe = json_normalize(venues)
Cafeteria_dataframe.head()

,id,name,categories,referralId,hasPerk,location.address,location.lat,location.lng,location.labeledLatLngs,location.distance,location.cc,location.city,location.state,location.country,location.formattedAddress,location.crossStreet,location.postalCode
0,4f9165bde4b04fef0087e4e2,Cafeteria,"[{'id': '4bf58dd8d48988d16e941735', 'name': 'F...",v-1578118085,False,The Hospital for Sick Children (SickKids),43.657209,-79.386063,"[{'label': 'display', 'lat': 43.65720930549997...",372,CA,Toronto,ON,Canada,"[The Hospital for Sick Children (SickKids), To...",NaN,NaN
1,528e99a211d262edc3708a6f,Stay Cafeteria 慢走,"[{'id': '4bf58dd8d48988d142941735', 'name': 'A...",v-1578118085,False,388 Spadina Ave,43.655454,-79.399163,"[{'label': 'display', 'lat': 43.65545427446646...",977,CA,Toronto,ON,Canada,"[388 Spadina Ave (at Nassau St), Toronto ON M5...",at Nassau St,M5T 2G5
2,53bd6dfe498e6c1d3bc9333e,Industrial Alliance Cafeteria,"[{'id': '4bf58dd8d48988d128941735', 'name': 'C...",v-1578118085,False,NaN,43.655657,-79.389443,"[{'label': 'display', 'lat': 43.655657, 'lng':...",260,CA,Toronto,ON,Canada,"[Toronto ON, Canada]",NaN,NaN
3,4ec2b68a8231a83de8c45191,OCAD Cafeteria,"[{'id': '4bf58dd8d48988d1a1941735', 'name': 'C...",v-1578118085,False,100 McCaul St.,43.652780,-79.391762,"[{'label': 'display', 'lat': 43.65278016892648...",389,CA,Toronto,ON,Canada,"[100 McCaul St. (Second Floor), Toronto ON, Ca...",Second Floor,NaN
4,556f826b498e07f469e6bf1f,Dears Cafeteria,"[{'id': '4bf58dd8d48988d1d0941735', 'name': 'D...",v-1578118085,False,NaN,43.654179,-79.392555,"[{'label': 'display', 'lat': 43.65417861938476...",431,CA,NaN,NaN,Canada,[Canada],NaN,NaN


## **Clean Cafeteria Dataframe**

In [33]:
# keep only columns that include venue name, and anything that is associated with location
Cafeteria_clean_columns = ['name', 'categories'] + [col for col in Cafeteria_dataframe.columns if col.startswith('location.')]+ ['id']
clean_Cafeteria_dataframe = Cafeteria_dataframe.loc[:,Cafeteria_clean_columns]

# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list4 = row['categories']
    except:
        categories_list4 = row['venue.categories']
        
    if len(categories_list4) == 0:
        return None
    else:
        return categories_list4[0]['name']

# filter the category for each row
clean_Cafeteria_dataframe['categories'] = clean_Cafeteria_dataframe.apply(get_category_type, axis=1)

# clean column names by keeping only last term
clean_Cafeteria_dataframe.columns = [column.split('.')[-1] for column in clean_Cafeteria_dataframe.columns]

clean_Cafeteria_dataframe.head()

,name,categories,address,lat,lng,labeledLatLngs,distance,cc,city,state,country,formattedAddress,crossStreet,postalCode,id
0,Cafeteria,Fast Food Restaurant,The Hospital for Sick Children (SickKids),43.657209,-79.386063,"[{'label': 'display', 'lat': 43.65720930549997...",372,CA,Toronto,ON,Canada,"[The Hospital for Sick Children (SickKids), To...",NaN,NaN,4f9165bde4b04fef0087e4e2
1,Stay Cafeteria 慢走,Asian Restaurant,388 Spadina Ave,43.655454,-79.399163,"[{'label': 'display', 'lat': 43.65545427446646...",977,CA,Toronto,ON,Canada,"[388 Spadina Ave (at Nassau St), Toronto ON M5...",at Nassau St,M5T 2G5,528e99a211d262edc3708a6f
2,Industrial Alliance Cafeteria,Cafeteria,NaN,43.655657,-79.389443,"[{'label': 'display', 'lat': 43.655657, 'lng':...",260,CA,Toronto,ON,Canada,"[Toronto ON, Canada]",NaN,NaN,53bd6dfe498e6c1d3bc9333e
3,OCAD Cafeteria,College Cafeteria,100 McCaul St.,43.652780,-79.391762,"[{'label': 'display', 'lat': 43.65278016892648...",389,CA,Toronto,ON,Canada,"[100 McCaul St. (Second Floor), Toronto ON, Ca...",Second Floor,NaN,4ec2b68a8231a83de8c45191
4,Dears Cafeteria,Dessert Shop,NaN,43.654179,-79.392555,"[{'label': 'display', 'lat': 43.65417861938476...",431,CA,NaN,NaN,Canada,[Canada],NaN,NaN,556f826b498e07f469e6bf1f


In [34]:
# delete unnecessary columns
clean_Cafeteria_dataframe2= clean_Cafeteria_dataframe.drop(['cc', 'city', 'country', 'crossStreet', 'distance', 'formattedAddress',\
                                        'labeledLatLngs', 'id'], axis=1)
clean_Cafeteria_dataframe2

,name,categories,address,lat,lng,state,postalCode
0,Cafeteria,Fast Food Restaurant,The Hospital for Sick Children (SickKids),43.657209,-79.386063,ON,NaN
1,Stay Cafeteria 慢走,Asian Restaurant,388 Spadina Ave,43.655454,-79.399163,ON,M5T 2G5
2,Industrial Alliance Cafeteria,Cafeteria,NaN,43.655657,-79.389443,ON,NaN
3,OCAD Cafeteria,College Cafeteria,100 McCaul St.,43.652780,-79.391762,ON,NaN
4,Dears Cafeteria,Dessert Shop,NaN,43.654179,-79.392555,NaN,NaN
5,Chestnut Tree Cafeteria,Cafeteria,89 Chestnut Street,43.648231,-79.384045,ON,NaN
6,Sears Cafeteria,Food Court,Yonge,43.656038,-79.380672,ON,NaN
7,De La Salle College Cafeteria,College Cafeteria,131 Farnham Avenue,43.683003,-79.397815,ON,M4V 1H7
8,Med Sci Cafeteria,College Cafeteria,NaN,43.660260,-79.392977,ON,NaN
9,Robarts Cafeteria,College Cafeteria,130 St. George St.,43.664864,-79.399766,ON,NaN


In [35]:
# delete rows with none values
df_Cafeteria = clean_Cafeteria_dataframe2.dropna(axis=0, how='any', thresh=None, subset=None, inplace=False)
df_Cafeteria

,name,categories,address,lat,lng,state,postalCode
1,Stay Cafeteria 慢走,Asian Restaurant,388 Spadina Ave,43.655454,-79.399163,ON,M5T 2G5
7,De La Salle College Cafeteria,College Cafeteria,131 Farnham Avenue,43.683003,-79.397815,ON,M4V 1H7
19,Marketeria,Restaurant,30 Bond St.,43.653585,-79.378843,ON,M5B 1W8
22,The Hub,College Cafeteria,350 Victoria St.,43.658585,-79.380622,ON,M5B 2K3
27,6th Floor Cafeteria 12 Concorde Place,Café,12 Concorde Place,43.721861,-79.329339,ON,m3c 3k7
28,Innis College,Student Center,2 Sussex Ave,43.665556,-79.399298,ON,M5S 1J5


## **Search for Shopping Stores**

In [36]:
# search for Shopping
search_query = 'Shopping'
radius = 1000

# Define the corresponding URL
url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(ClIENT_ID, ClIENT_SECRET, latitude, longitude, VERSION, search_query, radius, LIMIT)
url

'https://api.foursquare.com/v2/venues/search?client_id=TGM5K1WCBLTUMURGV1KRVXHNJERYE5WKFOZFW3ET2MP5OL1S&client_secret=A1Y33Z44WSTFEL4XASW20OMOWGIHQY1NO0QXXCTWUXP4ANPV&ll=43.653963,-79.387207&v=20200104&query=Shopping&radius=1000&limit=30'

In [0]:
# Send the GET Request and examine the results
sresults = requests.get(url).json()
#sresults

In [38]:
# assign relevant part of JSON to venues
venues = sresults['response']['venues']

# tranform venues into a dataframe
Shopping_dataframe = json_normalize(venues)
Shopping_dataframe.head()

,id,name,categories,referralId,hasPerk,location.address,location.crossStreet,location.lat,location.lng,location.labeledLatLngs,location.distance,location.postalCode,location.cc,location.city,location.state,location.country,location.formattedAddress,location.neighborhood
0,4ad4c062f964a520fcf720e3,Dragon City Shopping Mall 龍城,"[{'id': '4bf58dd8d48988d1fd941735', 'name': 'S...",v-1578118112,False,280 Spadina Ave,at Dundas St W,43.652774,-79.398222,"[{'label': 'display', 'lat': 43.65277367198862...",897,M5T 3A5,CA,Toronto,ON,Canada,"[280 Spadina Ave (at Dundas St W), Toronto ON ...",NaN
1,4ad77a12f964a520260b21e3,CF Toronto Eaton Centre,"[{'id': '4bf58dd8d48988d1fd941735', 'name': 'S...",v-1578118112,False,220 Yonge St,btwn Queen & Dundas,43.653594,-79.380611,NaN,532,M5B 2H1,CA,Toronto,ON,Canada,"[220 Yonge St (btwn Queen & Dundas), Toronto O...","Downtown Toronto, Toronto, ON"
2,4b9e86dff964a520e6eb36e3,TD Centre Shopping Concourse,"[{'id': '4bf58dd8d48988d1fd941735', 'name': 'S...",v-1578118112,False,66 Wellington St W,btw York & Bay St,43.647184,-79.380932,"[{'label': 'display', 'lat': 43.64718424611984...",908,M5K 1A1,CA,Toronto,ON,Canada,"[66 Wellington St W (btw York & Bay St), Toron...",NaN
3,5840cafe06f1a34af80cc609,Saks Fifth Avenue Club - Personal Shopping,"[{'id': '4bf58dd8d48988d1f6941735', 'name': 'D...",v-1578118112,False,176 Yonge Street,Queen Street West,43.651810,-79.379192,"[{'label': 'display', 'lat': 43.65181, 'lng': ...",688,M5C 2L7,CA,Toronto,ON,Canada,"[176 Yonge Street (Queen Street West), Toronto...",NaN
4,4b83d653f964a520d91231e3,Yellow Pages - RedFlagDeals.com,"[{'id': '4bf58dd8d48988d124941735', 'name': 'O...",v-1578118112,False,461 King St. West,NaN,43.647958,-79.396006,"[{'label': 'display', 'lat': 43.64795765580337...",974,M5V 1K4,CA,Toronto,ON,Canada,"[461 King St. West, Toronto ON M5V 1K4, Canada]",NaN


## **Clean Shopping Dataframe**

In [39]:
# keep only columns that include venue name, and anything that is associated with location
Shopping_clean_columns = ['name', 'categories'] + [col for col in Shopping_dataframe.columns if col.startswith('location.')]+ ['id']
clean_Shopping_dataframe = Shopping_dataframe.loc[:,Shopping_clean_columns]

# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list5 = row['categories']
    except:
        categories_list5 = row['venue.categories']
        
    if len(categories_list5) == 0:
        return None
    else:
        return categories_list5[0]['name']

# filter the category for each row
clean_Shopping_dataframe['categories'] = clean_Shopping_dataframe.apply(get_category_type, axis=1)

# clean column names by keeping only last term
clean_Shopping_dataframe.columns = [column.split('.')[-1] for column in clean_Shopping_dataframe.columns]

clean_Shopping_dataframe.head()

,name,categories,address,crossStreet,lat,lng,labeledLatLngs,distance,postalCode,cc,city,state,country,formattedAddress,neighborhood,id
0,Dragon City Shopping Mall 龍城,Shopping Mall,280 Spadina Ave,at Dundas St W,43.652774,-79.398222,"[{'label': 'display', 'lat': 43.65277367198862...",897,M5T 3A5,CA,Toronto,ON,Canada,"[280 Spadina Ave (at Dundas St W), Toronto ON ...",NaN,4ad4c062f964a520fcf720e3
1,CF Toronto Eaton Centre,Shopping Mall,220 Yonge St,btwn Queen & Dundas,43.653594,-79.380611,NaN,532,M5B 2H1,CA,Toronto,ON,Canada,"[220 Yonge St (btwn Queen & Dundas), Toronto O...","Downtown Toronto, Toronto, ON",4ad77a12f964a520260b21e3
2,TD Centre Shopping Concourse,Shopping Mall,66 Wellington St W,btw York & Bay St,43.647184,-79.380932,"[{'label': 'display', 'lat': 43.64718424611984...",908,M5K 1A1,CA,Toronto,ON,Canada,"[66 Wellington St W (btw York & Bay St), Toron...",NaN,4b9e86dff964a520e6eb36e3
3,Saks Fifth Avenue Club - Personal Shopping,Department Store,176 Yonge Street,Queen Street West,43.651810,-79.379192,"[{'label': 'display', 'lat': 43.65181, 'lng': ...",688,M5C 2L7,CA,Toronto,ON,Canada,"[176 Yonge Street (Queen Street West), Toronto...",NaN,5840cafe06f1a34af80cc609
4,Yellow Pages - RedFlagDeals.com,Office,461 King St. West,NaN,43.647958,-79.396006,"[{'label': 'display', 'lat': 43.64795765580337...",974,M5V 1K4,CA,Toronto,ON,Canada,"[461 King St. West, Toronto ON M5V 1K4, Canada]",NaN,4b83d653f964a520d91231e3


In [40]:
# delete unnecessary columns
clean_Shopping_dataframe2= clean_Shopping_dataframe.drop(['cc', 'city', 'country', 'crossStreet', 'distance', 'formattedAddress',\
                                        'labeledLatLngs', 'neighborhood', 'id'], axis=1)
clean_Shopping_dataframe2

,name,categories,address,lat,lng,postalCode,state
0,Dragon City Shopping Mall 龍城,Shopping Mall,280 Spadina Ave,43.652774,-79.398222,M5T 3A5,ON
1,CF Toronto Eaton Centre,Shopping Mall,220 Yonge St,43.653594,-79.380611,M5B 2H1,ON
2,TD Centre Shopping Concourse,Shopping Mall,66 Wellington St W,43.647184,-79.380932,M5K 1A1,ON
3,Saks Fifth Avenue Club - Personal Shopping,Department Store,176 Yonge Street,43.651810,-79.379192,M5C 2L7,ON
4,Yellow Pages - RedFlagDeals.com,Office,461 King St. West,43.647958,-79.396006,M5V 1K4,ON


In [41]:
# delete rows which its category is not Shopping Mall
df_Shopping = clean_Shopping_dataframe2[clean_Shopping_dataframe2.categories == 'Shopping Mall']
df_Shopping

,name,categories,address,lat,lng,postalCode,state
0,Dragon City Shopping Mall 龍城,Shopping Mall,280 Spadina Ave,43.652774,-79.398222,M5T 3A5,ON
1,CF Toronto Eaton Centre,Shopping Mall,220 Yonge St,43.653594,-79.380611,M5B 2H1,ON
2,TD Centre Shopping Concourse,Shopping Mall,66 Wellington St W,43.647184,-79.380932,M5K 1A1,ON


## **Generate map to visualize hotels, shopping stores and Cafeteria and how they cluster together**

In [42]:
# create dataframe of hotels, shopping stores and Cafeteria
hotel_neighbourhood_df = pd.concat([df_hotel, df_Cafeteria, df_Shopping], ignore_index=True)
hotel_neighbourhood_df

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  """Entry point for launching an IPython kernel.


,address,categories,lat,lng,name,postalCode,state
0,123 Queen Street West,Hotel,43.651144,-79.384329,Sheraton Centre Toronto Hotel,M5H 2M9,ON
1,123 Queen St. W,Event Space,43.651217,-79.383771,Grand Ballroom,M5H 2M9,ON
2,388 Spadina Ave,Asian Restaurant,43.655454,-79.399163,Stay Cafeteria 慢走,M5T 2G5,ON
3,131 Farnham Avenue,College Cafeteria,43.683003,-79.397815,De La Salle College Cafeteria,M4V 1H7,ON
4,30 Bond St.,Restaurant,43.653585,-79.378843,Marketeria,M5B 1W8,ON
5,350 Victoria St.,College Cafeteria,43.658585,-79.380622,The Hub,M5B 2K3,ON
6,12 Concorde Place,Café,43.721861,-79.329339,6th Floor Cafeteria 12 Concorde Place,m3c 3k7,ON
7,2 Sussex Ave,Student Center,43.665556,-79.399298,Innis College,M5S 1J5,ON
8,280 Spadina Ave,Shopping Mall,43.652774,-79.398222,Dragon City Shopping Mall 龍城,M5T 3A5,ON
9,220 Yonge St,Shopping Mall,43.653594,-79.380611,CF Toronto Eaton Centre,M5B 2H1,ON


In [43]:
# Generate map to visualize hotel neighbourhood including shopping stores and Cafeteria 
hotel_map = folium.Map(location=[latitude, longitude], zoom_start=14)

for lat, lng, name, categories, address in zip(hotel_neighbourhood_df['lat'], hotel_neighbourhood_df['lng'], 
                                           hotel_neighbourhood_df['name'], hotel_neighbourhood_df['categories'],\
                                               hotel_neighbourhood_df['address']):
    label = '{}, {}'.format(name, address)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='blue',
        fill_opacity=0.7,
        parse_html=False).add_to(hotel_map)  
    
hotel_map

## **Generate map to visualize Park, Restaurant and Cafeteria and how they cluster together**

In [44]:
# create dataframe of Park, Restaurant and Cafeteria
park_neighbourhood_df = pd.concat([df_park, df_Restaurant, df_Cafeteria,], ignore_index=True)
park_neighbourhood_df

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  """Entry point for launching an IPython kernel.


,address,categories,lat,lng,name,postalCode,state
0,University Ave.,Park,43.663946,-79.392180,Queen's Park,M5R 2E8,ON
1,1053 Dundas St. W.,Park,43.647072,-79.413756,Trinity Bellwoods Park,M5H 2N2,ON
2,btwn Bellevue & Augusta Ave.,Park,43.653610,-79.402199,Bellevue Square Park,M5T 2N4,ON
3,50 Fort York Blvd,Park,43.638762,-79.397067,Canoe Landing Park,M5V 3Z1,ON
4,550 Broadview Ave,Park,43.669951,-79.355493,Riverdale Park East,M4K 2P1,ON
5,519 Church St,Park,43.666879,-79.381068,Barbara Hall Park,M4Y 2K9,ON
6,110 Chestnut Street,American Restaurant,43.654884,-79.385931,Hemispheres Restaurant & Bistro,M5G 1R3,ON
7,323 Spadina Ave.,Dim Sum Restaurant,43.654318,-79.398650,Rol San Restaurant 龍笙棧,M5T 2E9,ON
8,309 Spadina Ave.,Chinese Restaurant,43.653866,-79.398334,Swatow Restaurant 汕頭小食家,M5T 2E6,ON
9,353 Spadina Ave.,Chinese Restaurant,43.655337,-79.398897,New Sky Restaurant 小沙田食家,M5T 2G3,ON


In [45]:
# Generate map to visualize park neighbourhood including Restaurant and Cafeteria 
park_map = folium.Map(location=[latitude, longitude], zoom_start=14)

for lat, lng, name, categories, address in zip(park_neighbourhood_df['lat'], park_neighbourhood_df['lng'], 
                                           park_neighbourhood_df['name'], park_neighbourhood_df['categories'],\
                                               park_neighbourhood_df['address']):
    label = '{}, {}'.format(name, address)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='blue',
        fill_opacity=0.7,
        parse_html=False).add_to(park_map)  
    
park_map

In [46]:
print("Thank you for visiting our project!\a")

Thank you for visiting our project!
